<a href="https://colab.research.google.com/github/PhilipPfeffer/haptic_vest/blob/main/customnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
from numpy import loadtxt
import keras
from keras import layers
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
import os
import shutil
import math

# Get Dataset

In [5]:
# Definitions
dataset ='/content/drive/My Drive/Haptic Vest/dataset_person_car'
classes = ['neither', 'person', None , 'car']
all_processed_imgs_dir = '/content/drive/My Drive/Haptic Vest/person_car_other_imgs_processed'

In [6]:
train_ids = np.loadtxt(dataset, dtype=(int, int))
train_ids_dict = dict(train_ids)

In [7]:
x_train_ids, y_train_ids = train_ids[:,0], train_ids[:,1]
assert(x_train_ids.shape == y_train_ids.shape)
x_train_ids[0], y_train_ids[0]
print(f"There are {len(x_train_ids)} images, set(x_train_ids) = {len(set(x_train_ids))} images.")

There are 77650 images, set(x_train_ids) = 77033 images.


In [9]:
######################### 
# CHANGE HERE
######################### 
num_examples_name = '40000'
######################### 

dataset_X_parent = '/content/dataset_' + num_examples_name
dataset_X = '/content/dataset_' + num_examples_name + '/processed_subset' + num_examples_name
# os.mkdir(dataset_X_parent)  # Uncomment if directory doesn't exist
!unzip /content/processed_subset40000.zip -d dataset_40000

Streaming output truncated to the last 5000 lines.
  inflating: dataset_40000/__MACOSX/processed_subset40000/._000000330611.jpg  
  inflating: dataset_40000/processed_subset40000/000000431026.jpg  
  inflating: dataset_40000/__MACOSX/processed_subset40000/._000000431026.jpg  
  inflating: dataset_40000/processed_subset40000/000000581929.jpg  
  inflating: dataset_40000/__MACOSX/processed_subset40000/._000000581929.jpg  
  inflating: dataset_40000/processed_subset40000/000000107011.jpg  
  inflating: dataset_40000/__MACOSX/processed_subset40000/._000000107011.jpg  
  inflating: dataset_40000/processed_subset40000/000000319283.jpg  
  inflating: dataset_40000/__MACOSX/processed_subset40000/._000000319283.jpg  
  inflating: dataset_40000/processed_subset40000/000000081274.jpg  
  inflating: dataset_40000/__MACOSX/processed_subset40000/._000000081274.jpg  
  inflating: dataset_40000/processed_subset40000/000000226701.jpg  
  inflating: dataset_40000/__MACOSX/processed_subset40000/._0000002

## Set sizes

In [10]:
class_ids = [0,1,3]

In [11]:
dataset_X_ids = {}
dataset_X_labels = []
for entry in os.scandir(dataset_X):
  if entry.is_dir() or os.path.basename(entry)[0] == '.':
    continue
  id = int(os.path.basename(entry)[:-4])
  dataset_X_ids[id] = train_ids_dict[id]
  dataset_X_labels.append(train_ids_dict[id])

print(len(dataset_X_ids))
print(len(dataset_X_labels))

65534
65534


In [12]:
dev_pct = 0.1
test_pct = 0.05
train_pct = 1 - dev_pct - test_pct
assert (dev_pct + test_pct + train_pct == 1)

In [13]:
num_neither = np.sum(np.array(dataset_X_labels)==0)
num_person = np.sum(np.array(dataset_X_labels)==1)
num_car = np.sum(np.array(dataset_X_labels)==3)
num_class_examples = {0:num_neither, 1:num_person, 3: num_car}
print(num_class_examples, num_neither +num_person + num_car)

{0: 37585, 1: 25470, 3: 2479} 65534


In [14]:
num_test = {key: int(test_pct * num_class) for key, num_class in num_class_examples.items()}
num_dev = {key: int(dev_pct * num_class) for key, num_class in num_class_examples.items()}
num_train = {class_id: num_class_examples[class_id] - num_dev[class_id] - num_test[class_id] for class_id in class_ids}

print(num_test, "\n", num_dev, "\n", num_train)

{0: 1879, 1: 1273, 3: 123} 
 {0: 3758, 1: 2547, 3: 247} 
 {0: 31948, 1: 21650, 3: 2109}


In [15]:
# In case of fuck up, reset folders
!rm -r /content/dataset_40000/processed_subset40000/dev
!rm -r /content/dataset_40000/processed_subset40000/test
!rm -r /content/dataset_40000/processed_subset40000/train

!mkdir /content/dataset_40000/processed_subset40000/dev
!mkdir /content/dataset_40000/processed_subset40000/dev/0
!mkdir /content/dataset_40000/processed_subset40000/dev/1
!mkdir /content/dataset_40000/processed_subset40000/dev/3

!mkdir /content/dataset_40000/processed_subset40000/test
!mkdir /content/dataset_40000/processed_subset40000/test/0
!mkdir /content/dataset_40000/processed_subset40000/test/1
!mkdir /content/dataset_40000/processed_subset40000/test/3

!mkdir /content/dataset_40000/processed_subset40000/train
!mkdir /content/dataset_40000/processed_subset40000/train/0
!mkdir /content/dataset_40000/processed_subset40000/train/1
!mkdir /content/dataset_40000/processed_subset40000/train/3

rm: cannot remove '/content/dataset_40000/processed_subset40000/dev': No such file or directory
rm: cannot remove '/content/dataset_40000/processed_subset40000/test': No such file or directory
rm: cannot remove '/content/dataset_40000/processed_subset40000/train': No such file or directory


In [16]:
def path_to_id(path):
  return int(os.path.basename(entry)[:-4])

In [17]:
num_test_done = {class_id: 0 for class_id in class_ids}
num_dev_done = {class_id: 0 for class_id in class_ids}
num_train_done = {class_id: 0 for class_id in class_ids}
for idx, entry in enumerate(os.scandir(dataset_X)):
  if entry.is_dir() or os.path.basename(entry)[0] == '.':
    continue
  id = path_to_id(entry)
  label = train_ids_dict[id]
  
  if num_train_done[label] != num_train[label]:
    num_train_done[label] += 1
    shutil.copyfile(entry, dataset_X + '/train/' + str(label) + '/' + os.path.basename(entry))
  elif num_dev_done[label] != num_dev[label]:
    num_dev_done[label] += 1
    shutil.copyfile(entry, dataset_X + '/dev/' + str(label) + '/' + os.path.basename(entry))
  elif num_test_done[label] != num_test[label]:
    num_test_done[label] += 1
    shutil.copyfile(entry, dataset_X + '/test/' + str(label) + '/' + os.path.basename(entry))
  
  if idx % 5000 == 0:
    print(idx, num_train_done, num_dev_done, num_test_done)

0 {0: 1, 1: 0, 3: 0} {0: 0, 1: 0, 3: 0} {0: 0, 1: 0, 3: 0}
5000 {0: 2913, 1: 1915, 3: 173} {0: 0, 1: 0, 3: 0} {0: 0, 1: 0, 3: 0}
10000 {0: 5756, 1: 3888, 3: 357} {0: 0, 1: 0, 3: 0} {0: 0, 1: 0, 3: 0}
15000 {0: 8593, 1: 5849, 3: 558} {0: 0, 1: 0, 3: 0} {0: 0, 1: 0, 3: 0}
20000 {0: 11499, 1: 7745, 3: 755} {0: 0, 1: 0, 3: 0} {0: 0, 1: 0, 3: 0}
25000 {0: 14358, 1: 9682, 3: 958} {0: 0, 1: 0, 3: 0} {0: 0, 1: 0, 3: 0}
30000 {0: 17231, 1: 11630, 3: 1137} {0: 0, 1: 0, 3: 0} {0: 0, 1: 0, 3: 0}
35000 {0: 20048, 1: 13638, 3: 1312} {0: 0, 1: 0, 3: 0} {0: 0, 1: 0, 3: 0}
40000 {0: 22957, 1: 15541, 3: 1500} {0: 0, 1: 0, 3: 0} {0: 0, 1: 0, 3: 0}
45000 {0: 25811, 1: 17493, 3: 1694} {0: 0, 1: 0, 3: 0} {0: 0, 1: 0, 3: 0}
50000 {0: 28669, 1: 19422, 3: 1907} {0: 0, 1: 0, 3: 0} {0: 0, 1: 0, 3: 0}
55000 {0: 31500, 1: 21404, 3: 2094} {0: 0, 1: 0, 3: 0} {0: 0, 1: 0, 3: 0}
60000 {0: 31948, 1: 21650, 3: 2109} {0: 2408, 1: 1723, 3: 159} {0: 0, 1: 0, 3: 0}
65000 {0: 31948, 1: 21650, 3: 2109} {0: 3758, 1: 2547, 3: 2

In [18]:
print(num_train_done, num_dev_done, num_test_done)

{0: 31948, 1: 21650, 3: 2109} {0: 3758, 1: 2547, 3: 247} {0: 1879, 1: 1273, 3: 123}


In [19]:
# create generator
datagen_X = ImageDataGenerator()
# prepare an iterators for each dataset
train_it = datagen_X.flow_from_directory(dataset_X +'/train/', class_mode='categorical', color_mode='grayscale', target_size=(96,96))
dev_it = datagen_X.flow_from_directory(dataset_X +'/dev/', class_mode='categorical', color_mode='grayscale', target_size=(96,96))
test_it = datagen_X.flow_from_directory(dataset_X +'/test/', class_mode='categorical', color_mode='grayscale', target_size=(96,96))
# confirm the iterator works
batchX, batchy = train_it.next()
print('Batch shape=%s, min=%.3f, max=%.3f' % (batchX.shape, batchX.min(), batchX.max()))

Found 55707 images belonging to 3 classes.
Found 6552 images belonging to 3 classes.
Found 3275 images belonging to 3 classes.
Batch shape=(32, 96, 96, 1), min=0.000, max=255.000


# GPU Setup

In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
# CHECK THE GPU IS WORKING

# %tensorflow_version 2.x
# import tensorflow as tf
# import timeit

# device_name = tf.test.gpu_device_name()
# if device_name != '/device:GPU:0':
#   print(
#       '\n\nThis error most likely means that this notebook is not '
#       'configured to use a GPU.  Change this in Notebook Settings via the '
#       'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
#   raise SystemError('GPU device not found')

# def cpu():
#   with tf.device('/cpu:0'):
#     random_image_cpu = tf.random.normal((100, 100, 100, 3))
#     net_cpu = tf.keras.layers.Conv2D(32, 7)(random_image_cpu)
#     return tf.math.reduce_sum(net_cpu)

# def gpu():
#   with tf.device('/device:GPU:0'):
#     random_image_gpu = tf.random.normal((100, 100, 100, 3))
#     net_gpu = tf.keras.layers.Conv2D(32, 7)(random_image_gpu)
#     return tf.math.reduce_sum(net_gpu)
  
# # We run each op once to warm up; see: https://stackoverflow.com/a/45067900
# cpu()
# gpu()

# # Run the op several times.
# print('Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images '
#       '(batch x height x width x channel). Sum of ten runs.')
# print('CPU (s):')
# cpu_time = timeit.timeit('cpu()', number=10, setup="from __main__ import cpu")
# print(cpu_time)
# print('GPU (s):')
# gpu_time = timeit.timeit('gpu()', number=10, setup="from __main__ import gpu")
# print(gpu_time)
# print('GPU speedup over CPU: {}x'.format(int(cpu_time/gpu_time)))

Time (s) to convolve 32x7x7x3 filter over random 100x100x100x3 images (batch x height x width x channel). Sum of ten runs.
CPU (s):
3.3263175350000154
GPU (s):
0.10538961899999322
GPU speedup over CPU: 31x


In [41]:
with tf.device('/device:GPU:0'):
  model = keras.Sequential(
    [
        keras.Input(shape=(batchX.shape[1],batchX.shape[2], 1)),  # 96x96 greyscale image
        layers.Conv2D(filters=8, kernel_size=3, strides=1, padding='same', activation="relu"),
        layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='same', activation="relu"),
        layers.Conv2D(filters=16, kernel_size=3, strides=1, padding='same', activation="relu"),
        layers.MaxPool2D(),
        layers.Conv2D(filters=32, kernel_size=3, strides=2, padding='same', activation="relu"),        # layers.MaxPool2D(),
        layers.Conv2D(filters=64, kernel_size=3, strides=2, padding='same', activation="relu"),
        layers.MaxPool2D(),
        layers.Flatten(name='flatten'),
        Dense(100, activation='relu'),
        Dense(100, activation='relu'),
        Dense(64, activation='relu'),
        Dense(3, activation='softmax')
    ]
)

In [42]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_62 (Conv2D)           (None, 96, 96, 8)         80        
_________________________________________________________________
conv2d_63 (Conv2D)           (None, 96, 96, 16)        1168      
_________________________________________________________________
conv2d_64 (Conv2D)           (None, 96, 96, 16)        2320      
_________________________________________________________________
max_pooling2d_13 (MaxPooling (None, 48, 48, 16)        0         
_________________________________________________________________
conv2d_65 (Conv2D)           (None, 24, 24, 32)        4640      
_________________________________________________________________
conv2d_66 (Conv2D)           (None, 12, 12, 64)        18496     
_________________________________________________________________
max_pooling2d_14 (MaxPooling (None, 6, 6, 64)         

In [36]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [38]:
with tf.device('/device:GPU:0'):
  model.fit(train_it, epochs=20, batch_size=batchX.shape[0], validation_data=dev_it, validation_batch_size=batchX.shape[0])

Epoch 1/20
1741/1741 [==============================] - 27s 16ms/step - loss: 0.7334 - accuracy: 0.6500 - val_loss: 0.6910 - val_accuracy: 0.6763
Epoch 2/20
1741/1741 [==============================] - 27s 16ms/step - loss: 0.6758 - accuracy: 0.6906 - val_loss: 0.6642 - val_accuracy: 0.7056
Epoch 3/20
1741/1741 [==============================] - 27s 16ms/step - loss: 0.6522 - accuracy: 0.7081 - val_loss: 0.6541 - val_accuracy: 0.7092
Epoch 4/20
1741/1741 [==============================] - 28s 16ms/step - loss: 0.6228 - accuracy: 0.7290 - val_loss: 0.6560 - val_accuracy: 0.7129
Epoch 5/20
1741/1741 [==============================] - 27s 16ms/step - loss: 0.5899 - accuracy: 0.7457 - val_loss: 0.6617 - val_accuracy: 0.7108
Epoch 6/20
1741/1741 [==============================] - 27s 16ms/step - loss: 0.5514 - accuracy: 0.7665 - val_loss: 0.7085 - val_accuracy: 0.7085
Epoch 7/20
1741/1741 [==============================] - 27s 16ms/step - loss: 0.5086 - accuracy: 0.7875 - val_loss: 0.6892 -

KeyboardInterrupt: ignored

In [37]:
# evaluate the keras model
_, accuracy = model.evaluate(dev_it)
print('Accuracy: %.2f' % (accuracy*100))

205/205 [==============================] - 2s 12ms/step - loss: 2.9456 - accuracy: 0.0386
Accuracy: 3.86
